In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle

startDate = "20200424"
endDate = "20200506"
readPath = r'\\192.168.10.30\Kevin_zhenyu\rawData\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
opath = []
tpath = []
for i in dataPathLs:
    tpath.append(glob.glob(os.path.join(i, "mdTradeLog_***")))
tpath1 = tpath[::2]
tpath2 = tpath[1::2]
print(tpath1)
print(tpath2)
F1 = open("\\\\192.168.10.30\\Kevin_zhenyu\\index\\weight_table_CSI1000.pkl", 'rb')
df1 = pickle.load(F1)

for i in range(len(tpath1)):
    TradeLog = pd.read_csv(tpath2[i][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    TradeLog1 = pd.read_csv(tpath1[i][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    SH = TradeLog[TradeLog["exchId"] == 1]
    SH1 = TradeLog1[TradeLog1["exchId"] == 1]
    
    # completeness check
    sl1 = SH1["SecurityID"].unique()
    sl2 = SH["SecurityID"].unique()
    stocks = list(set(sl1) & set(sl2))
    SH = SH[SH["SecurityID"].isin(stocks)]
    SH1 = SH1[SH1["SecurityID"].isin(stocks)]
    
    d_ate = os.path.basename(tpath1[i][0]).split('_')[1]
    d_ate = d_ate[:4] + '-' + d_ate[4:6] + '-' + d_ate[6:]
    ID_list = df1[df1["date"] == d_ate]["ID"]
    len1 = ID_list[ID_list.str[:2] == "SH"].shape[0]
    len2 = ID_list[ID_list.str[:2] == "SZ"].shape[0]
    print(len1)
    print(len2)
    ID_list = ID_list.str[2:].astype(int)
    SH["idx"] = np.where(SH["SecurityID"].isin(ID_list.values), 1, 0)
    SH1["idx"] = np.where(SH1["SecurityID"].isin(ID_list.values), 1, 0)
       
    SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
    SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
    re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer")
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    display(os.path.basename(os.path.basename(tpath2[i][0]).split("_")[1] + "_TradeLog Check:"))
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo_y"])])
        print(len(re[np.isnan(re["sequenceNo_y"])]))
        print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
        display(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 1)])
        print(len(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 1)]))
        print(np.sort(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 1)]["TransactTime"].unique()))
        print(len(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 1)]["SecurityID"].unique()))
        print(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 1)]["SecurityID"].unique())
        display(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 0)])
        print(len(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 0)]))
        print(np.sort(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 0)]["TransactTime"].unique()))
        print(len(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 0)]["SecurityID"].unique()))
        print(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 0)]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo_x"])])
        print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
    
#     # ordering among stocks
#     SH1 = SH1.reset_index(drop=True)
#     SH1["innerSeq"] = SH1.index.values
#     re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="inner")
#     display(os.path.basename(os.path.basename(tpath2[i][0]).split("_")[1] + "_Ordering among stocks:"))
#     if (sorted(re["innerSeq"]) != re["innerSeq"]).any():
#         print("SHE Trade Data don't have same ordering among stocks")
#     else:
#         print("SHE Trade Data have same ordering among stocks")
#     print(len(re))
#     print(len(re[sorted(re["sequenceNo_y"]) != re["sequenceNo_y"]]))
    
#     # ordering within stocks
#     stocks = SH["SecurityID"].unique()
#     m = 0
#     diff = []
#     for i1 in stocks:
#         re1 = pd.merge(SH[SH["SecurityID"] == i1], SH1[SH1["SecurityID"] == i1], left_on=columns, right_on=columns, how="inner")
#         if (sorted(re1["innerSeq"]) != re1["innerSeq"]).any() == False:
#             m = m + 1
#         else:
#             diff.append(i1)
#     display(os.path.basename(os.path.basename(tpath2[i][0]).split("_")[1] + "_Ordering within stocks:"))
#     if m == len(stocks):
#         print("SHE Trade Data have same ordering within stocks")
#     else:
#         print("SHE Trade Data don't have same ordering within stocks")
    
#     del SH
#     del SH1
#     del re1
    


[['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200424_zs_92_01_day_data\\mdTradeLog_20200424_0840.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200427_zs_92_01_day_data\\mdTradeLog_20200427_0832.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200428_zs_92_01_day_data\\mdTradeLog_20200428_0832.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200429_zs_92_01_day_data\\mdTradeLog_20200429_0851.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200430_zs_92_01_day_data\\mdTradeLog_20200430_0847.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200506_zs_92_01_day_data\\mdTradeLog_20200506_0833.csv']]
[['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200424_zt_96_04_day_96data\\mdTradeLog_20200424_0841.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200427_zt_96_04_day_96data\\mdTradeLog_20200427_0835.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200428_zt_96_04_day_96data\\mdTradeLog_20200428_0846.csv'], ['\\\\192.1

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


360
640


'20200424_TradeLog Check:'

16235329
16235337
16235337
-----------------------------------------------


'baseline is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId_x,TransactTime,ApplSeqNum,SecurityID,ExecType,TradeBSFlag,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,idx_x,clockAtArrival_y,sequenceNo_y,exchId_y,idx_y
16235329,NaN,NaN,NaN,150000360,2999225,603095,F,N,295200,100,29520000,4824840,4791883,NaN,1587711638310865,121313744,1,0
16235330,NaN,NaN,NaN,150000360,2999226,603095,F,N,295200,200,59040000,4824840,4815217,NaN,1587711638310866,121313745,1,0
16235331,NaN,NaN,NaN,150000360,2999227,603095,F,N,295200,300,88560000,4824840,4827299,NaN,1587711638310867,121313746,1,0
16235332,NaN,NaN,NaN,150000360,2999228,603095,F,N,295200,200,59040000,4824840,4827444,NaN,1587711638310867,121313747,1,0
16235333,NaN,NaN,NaN,150000360,2999229,603095,F,N,295200,300,88560000,4824840,4828411,NaN,1587711638310868,121313748,1,0
16235334,NaN,NaN,NaN,150000360,2999230,603095,F,N,295200,200,59040000,4824840,4828426,NaN,1587711638310869,121313749,1,0
16235335,NaN,NaN,NaN,150000360,2999231,603095,F,N,295200,500,147600000,4824840,4829994,NaN,1587711638310870,121313750,1,0
16235336,NaN,NaN,NaN,150000360,2999232,603095,F,N,295200,1000,295200000,4824840,4830337,NaN,1587711638310870,121313751,1,0


[150000360]
1
[603095]


8

MemoryError: 

In [22]:
import pickle
import pandas as pd
import random
import numpy as np
import glob
import os

startDate = "20200506"
endDate = "20200506"
readPath = r'\\192.168.10.30\Kevin_zhenyu\rawData\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
opath = []
tpath = []
for i in dataPathLs:
    tpath.append(glob.glob(os.path.join(i, "mdTradeLog_***")))
tpath1 = tpath[::2]
tpath2 = tpath[1::2]
print(tpath1)
print(tpath2)

F1 = open("\\\\192.168.10.30\\Kevin_zhenyu\\index\\weight_table_CSI1000.pkl", 'rb')
df1 = pickle.load(F1)
df1[df1["date"] == "2020-05-06"]

d_ate = os.path.basename(tpath1[0][0]).split('_')[1]
d_ate = d_ate[:4] + '-' + d_ate[4:6] + '-' + d_ate[6:]
df1[df1["date"] == d_ate]["ID"].values

TradeLog = pd.read_csv(tpath2[0][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
TradeLog1 = pd.read_csv(tpath1[0][0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
SH = TradeLog[TradeLog["exchId"] == 1]
SH1 = TradeLog1[TradeLog1["exchId"] == 1]
sl1 = SH1["SecurityID"].unique()
sl2 = SH["SecurityID"].unique()
stocks = list(set(sl1) & set(sl2))
SH = SH[SH["SecurityID"].isin(stocks)]
SH1 = SH1[SH1["SecurityID"].isin(stocks)]

[['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200506_zs_92_01_day_data\\mdTradeLog_20200506_0833.csv']]
[['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200506_zt_96_04_day_96data\\mdTradeLog_20200506_0835.csv']]


In [27]:
ID_list = df1[df1["date"] == d_ate]["ID"]
len1 = ID_list[ID_list.str[:2] == "SH"].shape[0]
len2 = ID_list[ID_list.str[:2] == "SZ"].shape[0]
print(len1)
print(len2)
ID_list = ID_list.str[2:].astype(int)
display(ID_list)
SH["idx"] = np.where(SH["SecurityID"].isin(ID_list.values), 1, 0)
SH1["idx"] = np.where(SH1["SecurityID"].isin(ID_list.values), 1, 0)

360
640


733000      2185
733001      2405
733002    300661
733003    300450
733004    300558
           ...  
733995    603687
733996      2951
733997    603676
733998    603848
733999    603555
Name: ID, Length: 1000, dtype: int32

358

In [35]:
SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer")
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
display(os.path.basename(os.path.basename(tpath2[0][0]).split("_")[1] + "_TradeLog Check:"))
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
    display(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 1)])
    print(len(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 1)]))
    print(np.sort(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 1)]["TransactTime"].unique()))
    print(len(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 1)]["SecurityID"].unique()))
    print(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 1)]["SecurityID"].unique())
    display(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 0)])
    print(len(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 0)]))
    print(np.sort(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 0)]["TransactTime"].unique()))
    print(len(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 0)]["SecurityID"].unique()))
    print(re[(np.isnan(re["sequenceNo_y"])) & (re["idx_x"] == 0)]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

'test is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId_x,TransactTime,ApplSeqNum,SecurityID,ExecType,TradeBSFlag,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,idx_x,clockAtArrival_y,sequenceNo_y,exchId_y,idx_y
17328739,1588748428862188,130488389,1,150000680,2993964,688003,F,N,271500,200,54300000,4547018,4594100,0,NaN,NaN,NaN,NaN
17328740,1588748428862189,130488390,1,150000680,2993965,688003,F,N,271500,1700,461550000,4547018,4600609,0,NaN,NaN,NaN,NaN
17328741,1588748428862189,130488391,1,150000680,2993966,688003,F,N,271500,723,196294500,4547018,4492781,0,NaN,NaN,NaN,NaN
17328742,1588748428862190,130488392,1,150000680,2993967,688003,F,N,271500,100,27150000,4547018,4507748,0,NaN,NaN,NaN,NaN
17328743,1588748428862190,130488393,1,150000680,2993968,688003,F,N,271500,500,135750000,4547018,4537709,0,NaN,NaN,NaN,NaN
17328744,1588748428862192,130488394,1,150000680,2993969,688003,F,N,271500,772,209598000,4547018,4538003,0,NaN,NaN,NaN,NaN
17328745,1588748428862192,130488395,1,150000680,2993970,688003,F,N,271500,500,135750000,4547018,4568454,0,NaN,NaN,NaN,NaN
17328746,1588748428862194,130488396,1,150000680,2993971,688003,F,N,271500,300,81450000,4547018,4589312,0,NaN,NaN,NaN,NaN
17328747,1588748428862194,130488397,1,150000680,2993972,688003,F,N,271500,953,258739500,4547018,4454824,0,NaN,NaN,NaN,NaN
17328748,1588748428862195,130488398,1,150000680,2993973,688003,F,N,271500,8000,2172000000,4547018,4474152,0,NaN,NaN,NaN,NaN


45
[150000680]
1
[688003]


,clockAtArrival_x,sequenceNo_x,exchId_x,TransactTime,ApplSeqNum,SecurityID,ExecType,TradeBSFlag,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,idx_x,clockAtArrival_y,sequenceNo_y,exchId_y,idx_y


0
[]
0
[]
45
[150000680]
1
[688003]
